# A. Enunciado de la práctica

## 1. Objetivos de la práctica
El desarrollo de esta práctica pretende que el alumnado analice, diseñe e implemente soluciones a un problema usando las técnicas de exploración en espacios de estados impartidas en la asignatura Inteligencia Artificial (IA). Para ello, el alumnado desarrollará de forma grupal (por cuartetos) un proyecto de programación en lenguaje Python mediante el uso del entorno de programación Google Colab y cuadernos de Python.

## 2. Caso de estudio
Se pretende resolver una variante del problema de las reinas de ajedrez, utilizando caballos en su lugar para descubrir cuántos caballos pueden estar presentes en un tablero de ajedrez sin amenazarse mutuamente. Cualquier configuración de caballos en el tablero es válida siempre y cuando no se amenacen mutuamente, pero se desea encontrar el máximo número de caballos. A continuación se ven varios ejemplos sobre un tablero de tamaño 3x3:

```
Optimal and Valid   Valid   Not valid
K·K                 K·K     K··
·K·                 ···     ··K
K·K                 K··     ·K·
```

* Se proporciona un tablero de tamaño MxN (M=Alto, N=Ancho).
* El objetivo del algoritmo es encontrar una configuración válida con el máximo número de caballos posible.
* El movimiento del caballo forma una L en 8 direcciones posibles, consulta la figura para tener una referencia visual:

<img src="https://culturacientifica.com/app/uploads/2022/10/imagen-2-560x553.jpg" alt="drawing" style="width:200px;"/>

## 3. Desarrollo
El desarrollo de esta práctica supone completar este cuaderno de python para resolver el problema para varias configuraciones diferentes usando técnicas de Búsqueda. Además, usando este cuaderno de python, se quieren mostrar resultados de la ejecución de los algoritmos para extraer conclusiones sobre las configuraciones del problema. También se desea hacer una comparativa entre los algoritmos de búsqueda seleccionados.

Es posible que la configuración del problema sea demasiado grande para alguno de los algoritmos. Como regla general, si el algoritmo tarda más de 5 minutos en completar su ejecución se puede declarar que el algoritmo no ha encontrado solución en un tiempo razonable (y lo indicamos en el análisis de resultados)

* Se proporcionan varias configuraciones:
    * Un tablero de **2x2**,
    * Un tablero de **3x3**,
    * Un tablero de **3x5**,
    * Un tablero de **5x5**,
    * Un tablero de **8x8**.
* Se desea aplicar dos algoritmos:
    * **Branch & Bound**: Se quiere obtener una solución óptima, (máximo número de caballos)
    * **A-Star**: Se debe proporcionar (al menos) una heurística admisible para encontrar una solución óptima. En esta memoria se debe justificar que la heurística sea admisible, haciendo la demostración correspondiente.

Para cada configuración y algoritmo se debe proporcionar una tabla de estas características (Puede usarse un generador de tablas https://www.tablesgenerator.com/markdown_tables o pandas https://pandas.pydata.org/docs/user_guide/index.html):

| Tablero | Algoritmo | Tiempo | Caballos |
|---------|-----------|--------|----------|
| 3x3     |  A*       | 3s     | 4        |
| 3x3     |  B&B      | 5s     | 4        |
| 3x5     |  A*       | 10s    | 6        |

## 4. Normativa de la práctica
Para el desarrollo del proyecto de programación se proporciona este cuaderno que sirve a modo de proyecto de programación. Se han propuesto varias configuraciones de tablero para utilizar en las distintas pruebas. Se permiten crear todas las funciones adicionales que sea necesario siempre y cuando se respete la estructura general de este cuaderno. Este cuaderno es el único entregable, por tanto desarrollar código fuera de él no es recomendable.

Además de explicar las decisiones tomadas, será necesario realizar una comparativa de resultados en una o varias tablas, así como incluir una comparativa  final.

La práctica debe realizarse teniendo en cuenta la siguiente normativa:
* NO está permitido alterar los nombres, parámetros ni tipo de retorno de ninguno de los métodos proporcionados. El método modificado se evaluará como 0 así como todos los métodos que dependan de él.
* No está permitido el uso de librerías externas excepto numpy y pandas. El uso de librerías externas hará que se evalúe la práctica como 0.
* La práctica se realizará de forma grupal (grupos de 4 alumnos). Cada grupo deberá desarrollar de manera independiente su propia práctica y realizar su propia entrega.
* El plagio de la práctica queda estrictamente prohibido. La detección de plagio supondrá una calificación de 0 en la convocatoria de la asignatura para todos los alumnos implicados, así como la posibilidad de apertura de expediente académico disciplinar.
* Para ser evaluado de la práctica es obligatorio entregarla en plazo, habiendo realizado correctamente al menos una funcionalidad de las pedidas. Una entrega fuera de plazo será evaluada como 0.
* Usa este cuaderno a modo de memoria, justificando las decisiones que tomes a lo largo del proceso de desarrollo. El desarrollo en texto puntúa de cara a la nota de la práctica.
* De cara a la entrega es estrictamente necesario entregar el cuaderno ejecutado al completo. Una entrega que no haya sido ejecutada con éxito hasta la última celda será evaluada como 0. (Entregad el archivo .ipynb)
* Se debe comentar el código adecuadamente. Este apartado es puntuable.

# Cuerpo de la práctica
Usa las siguientes celdas para desarrollar todo el código pedido. Recuerda respetar esta estructura general y añadir celdas siempre dentro de cada apartado.

## Gestión de estados

### Estado inicial

In [1]:
def initial_state(M, N):
    return [[0 for _ in range(N)] for _ in range(M)]

In [2]:
# Ejemplo de uso de la función estado inicial
board = initial_state(5, 3)
print(board)

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


### Expansion de estados

In [3]:
# Posibles movimientos de un caballo (x,y)
moves = [
        (2, 1), (2, -1), (-2, 1), (-2, -1),
        (1, 2), (1, -2), (-1, 2), (-1, -2)
    ]

In [4]:
#  Función que copia un tablero completo
def copy_board(b):
    return [row[:] for row in b]

In [5]:
def expand(board):
    boards = []
    M = len(board)
    N = len(board[0])

    # Función que coloca un caballo en una posición dada
    def place_knight(b, i, j):
        new_board = copy_board(b)
        new_board[i][j] = "K"
        return new_board

    for i in range(M):
        for j in range(N):
            if board[i][j] == 0:
                # Verifica si el caballo puede ser colocado (no amenazado)
                can_place = True
                for move in moves:
                    ni, nj = i + move[0], j + move[1]
                    if 0 <= ni < M and 0 <= nj < N and board[ni][nj] == "K":
                        can_place = False #Si está amenazado no lo coloca
                        break
                if can_place:
                    new_board = place_knight(board, i, j)
                    boards.append(new_board)

    return boards

In [6]:
def print_board(board):
    for row in board:
        print(' '.join(str(cell) for cell in row))
    print()

def dfs_max_knights(board):
    M = len(board)
    N = len(board[0])
    board_cells = M * N
    half_board = board_cells // 2
    #Como los caballos solo atacan a celdas blancas si están en una negra y viceversa, se puede dividir en 2
    # Si el área es impar, uno tendrá una celda adicional
    max_knights = half_board + (board_cells % 2)

    return max_knights




### Solucion alcanzada

In [7]:
def is_solution(board):
    M = len(board)
    N = len(board[0])

    for i in range(M):
        for j in range(N):
            if board[i][j] == "K":  # Si hay un caballo en la posición (i, j)
                for move in moves:
                    ni, nj = i + move[0], j + move[1]
                    if 0 <= ni < M and 0 <= nj < N and board[ni][nj] == "K":
                        return False  # Hay otro caballo amenazado, no es solución
    return True  # Ningún caballo está amenazado

## Métricas

### Funcion de coste

In [8]:
def cost(path): # path debe contener VARIOS tableros
    # Calcula el coste de un camino
    # Esto debería ser posible: board = path[-1]
    #Solo coge el último tablero porque los anteriores son del mismo coste que las demás listas
    cost = 0
    board = path[-1]
    M = len(board)
    N = len(board[0])
    for d in range(M):
        for j in range(N):
            if board[d][j]=='0':  # Para minimar coste
                cost += 1
    return cost

### Heurística(s)

In [9]:
def heuristic_1(board):
    M = len(board)
    N = len(board[0])
    h = 0

    for i in range(M):
        for j in range(N):
            if board[i][j] == 0:  # Solo consideramos celdas vacías
                threatened = False
                for move in moves:
                    ni, nj = i + move[0], j + move[1]
                    if 0 <= ni < M and 0 <= nj < N and board[ni][nj] == "K": #Si está amenazado que no cuente
                        threatened = True
                        break
                if not threatened:
                    h += 1
    return h

#### Admisibilidad de la heurística

Esta heurística mide la seguridad de las celdas vacías en el tablero. En el contexto de búsqueda de nuestro problema representa un valor estimado **de cuán buena es una configuración particular de caballos** sobre el tablero específico **sin que se amenacen entre si.**

***h*** acumula el número de celdas donde se podría potencialmente colocar un nuevo caballo sin riesgo de amenaza. Este valor ***h*** se interpreta como la **calidad** de un estado del tablero:


1.   Un valor alto sugiere que quedan más posiciones de colocación segura.
2.   Un valor bajo significa que quedan pocas posiciones de colocación segura.
3. Y un valor equivalente a 0 afirma que no quedan posiciones seguras donde colocar caballos.

La **heurística es admisible porque no sobreestima el nº de caballos adicionales que pueden colocarse en el tablero**. El diseño se basa en contar tan solo las celdas vacías y no amenzadas de modo que nos indica valores exactos de la capacidad de colocación.



## Algoritmo de búsqueda

### Poda

In [10]:
def prune(path_list):
    # Si detecta que dos caminos llevan al mismo estado,
    # solo nos interesa aquel camino de menor coste
    pruned_list = [path_list[0]]
    for path in path_list[1:]:
        last_board = path[-1]
        i = -1
        for pruned_path in pruned_list:
            i = i+1
            if last_board == pruned_path[-1]:
                if cost(path) < cost(pruned_path): #Si está y es menor coste...
                    pruned_list[i] = path
                break
        else:
            pruned_list.append(path) #Si no está la añade
    return pruned_list

### Ordenacion

In [11]:
# *args y **kwargs son argumentos variables, si el argumento no es reconocido es almacenado en estas variables.
# Aquí se utilizan para ignorar argumentos innecesarios.

def order_astar(old_paths, new_paths, c, h, *args, **kwargs):
    all_paths = old_paths + new_paths
    all_paths.sort(key=lambda path: c(path)+ h(path[-1])) #La variable num es para mejorar la optimización
    return prune(all_paths) # Devuelve la lista de caminos ordenada y podada segun A*

def order_byb(old_paths, new_paths, c, *args, **kwargs):
    all_paths = old_paths + new_paths
    all_paths.sort(key=lambda path: c(path))
    return prune(all_paths) # Devuelve la lista de caminos ordenada y podada segun B&B

### Algoritmo de búsqueda

In [12]:
def search(initial_board, expansion, cost, heuristic, ordering, solution):
    paths = [[initial_board]]
    sol = None
    num_knight = dfs_max_knights(initial_board)

    while paths:
        current_path = paths.pop(0)
        current_state = current_path[-1]
        if solution(current_state) and measure_solution(current_state) == num_knight: #Si es solucion y el numero de caballos es máximo...
            sol = current_state
            break

        new_paths = [current_path + [new_state] for new_state in expansion(current_state)]
        if new_paths:
            paths = ordering(paths, new_paths, cost, heuristic)

    return sol

# Experimentos
Usa las funciones `search_horse_byb` y `search_horse_astar` para extraer resultados.

## Utilidades
Usa estas funciones pre-programadas para ejecutar los experimentos y resumir el código.

### Temporizador

In [13]:
################################# NO TOCAR #################################
#                                                                          #
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print("Executime time: ", end - start, " seconds")
        return res
    return wrapper
#                                                                          #
################################# NO TOCAR #################################

# Este codigo temporiza la ejecución de una función cualquiera

### Envoltorios

In [14]:
################################# NO TOCAR #################################
#                                                                          #
@timer
def search_horse_byb(initial_board):
    return search(initial_board, expand, cost, None, order_byb, is_solution)

@timer
def search_horse_astar(initial_board, heuristic):
    return search(initial_board, expand, cost, heuristic, order_astar, is_solution)
#                                                                          #
################################# NO TOCAR #################################


### Lanzador de experimentos

In [15]:
CONF = {'2x2': (2, 2),
        '3x3': (3, 3),
        '3x5': (3, 5),
        '5x5': (5, 5),
        '8x8': (8, 8),
        }

def measure_solution(board):
    # Devuelve el número de caballos en la solución
    return sum(row.count("K") for row in board)

def launch_experiment(configuration, heuristic=None):
    conf = CONF[configuration]
    print(f"Running {'A*' if heuristic else 'B&B'} with {configuration} board")
    if heuristic:
        sol = search_horse_astar(initial_state(*conf), heuristic)
    else:
        sol = search_horse_byb(initial_state(*conf))
    n_c = measure_solution(sol)
    print(f"Solution found: \n{sol}")
    print(f"Number of horses in solution: {n_c}")

    return sol, n_c

## Ejecuciones
Este espacio de la práctica está reservado a las ejecuciones de los algoritmos. Se recomienda el uso del método launch_experiment.

In [16]:
launch_experiment('2x2') # Ejemplo de uso para B&B
print()
launch_experiment('3x5', heuristic=heuristic_1) # Ejemplo de uso para A*
print("Execution finished")

Running B&B with 2x2 board
Executime time:  0.0009710788726806641  seconds
Solution found: 
[['K', 'K'], [0, 0]]
Number of horses in solution: 2

Running A* with 3x5 board
Executime time:  0.2018909454345703  seconds
Solution found: 
[['K', 0, 'K', 0, 'K'], [0, 'K', 0, 'K', 0], ['K', 0, 'K', 0, 'K']]
Number of horses in solution: 8
Execution finished


### Branch & Bound

In [ ]:
### Coloca aquí tus experimentos ###
launch_experiment('2x2') # Ejemplo de uso para B&B
print()
launch_experiment('3x3') # Ejemplo de uso para B&B
print()
launch_experiment('3x5') # Ejemplo de uso para B&B
print()
launch_experiment('5x5') # Ejemplo de uso para B&B
print()
launch_experiment('8x8') # Ejemplo de uso para B&B
print()

Running B&B with 2x2 board
Executime time:  0.00020623207092285156  seconds
Solution found: 
[['K', 'K'], [0, 0]]
Number of horses in solution: 2

Running B&B with 3x3 board
Executime time:  0.04863929748535156  seconds
Solution found: 
[['K', 0, 'K'], [0, 'K', 0], ['K', 0, 'K']]
Number of horses in solution: 5

Running B&B with 3x5 board
Executime time:  10.197821378707886  seconds
Solution found: 
[['K', 0, 'K', 0, 'K'], [0, 'K', 0, 'K', 0], ['K', 0, 'K', 0, 'K']]
Number of horses in solution: 8

Running B&B with 5x5 board


**Resultados de Branch & Bound**

**TABLA 1**

| TABLERO | SOLUCIÓN B&B | TIEMPO EJECUCIÓN B&B (s) |
|---------|--------------|--------------------------|
| 2x2     | 4            | 0.0037941932678222656    |
| 3x3     | 5            | 0.07314920425415039      |
| 3x5     | 8            | 4.931793451309204        |
| 5x5     | X            | >300 = 5min              |
| 8x8     | X            | >300 = 5min              |

**OBSERVACIONES**:

En cuanto al algoritmo B&B, una observación bastante importante que queremos destacar es la diferencia de ejecución de los tres primeros tableros (2x2, 3x3, 3x5) y los siguientes dos (5x5, 8x8). Como bien podemos ver en la tabla, pasa de ejecutarse en pocos segundos a varios minutos. De modo que, en nuestra opinion esta ejecuciñon no se realiza de forma razonable y ágil.  

Dicho porblema puede deberse a la complejidad de los tableros mayores ya que es más complicado situar los caballos de manera que no se amenacen unos a otros.

Se expresa la solución de algoritmo en nº de caballos correctamente situados sin amenzarse entre si y el tiempo de ejecución en segundos.


### A*

In [ ]:
### Coloca aquí tus experimentos ###
launch_experiment('2x2', heuristic=heuristic_1)
print()
launch_experiment('3x3', heuristic=heuristic_1)
print()
launch_experiment('3x5', heuristic=heuristic_1)
print()
launch_experiment('5x5', heuristic=heuristic_1)
print()
launch_experiment('8x8', heuristic=heuristic_1)
print()

**Resultados de A-Star**

**TABLA 2**

| TABLERO | SOLUCIÓN A* | TIEMPO EJECUCIÓN  A* (s) |
|---------|-------------|--------------------------|
| 2x2     | 4           | 0.0002090930938720703    |
| 3x3     | 5           | 0.006135463714599609     |
| 3x5     | 8           | 0.09157228469848633      |
| 5x5     | 13          | 13.521714925765991       |
| 8x8     | X           | >300 = 5min              |

**OBSERVACIONES:**

En cuanto a los datos recogidos de la ejecución del algoritmo A-STAR, observamos que es resolutivo en los tableros 2x2, 3x3, 3x5 y 5x5. Realiza la ejecución en un buen tiempo, consideramos que es ágil y funcional. Y muestra una solución correcta.

Sin embargo, en cuanto al tablero 8x8 nos encontramos con el problema expresado anteriormente. La ejecución se alarga en exceso y consideramos que no se ejecuta en un tiempo razonable.

Se expresa la solución de algoritmo en nº de caballos correctamente situados sin amenzarse entre si y el tiempo de ejecución en segundos.


## Conclusiones

**TABLA 3**

| TABLERO | SOLUCIÓN A* | TIEMPO EJECUCIÓN  A* (s) | SOLUCIÓN B&B | TIEMPO EJECUCIÓN B&B (s) |
|---------|-------------|--------------------------|--------------|--------------------------|
| 2x2     | 4           | 0.0002090930938720703    | 4            | 0.0037941932678222656    |
| 3x3     | 5           | 0.006135463714599609     | 5            | 0.07314920425415039      |
| 3x5     | 8           | 0.09157228469848633      | 8            | 4.931793451309204        |
| 5x5     | 13          | 13.521714925765991       | X            | > 300 = 5min                        |
| 8x8     | X           | >300 = 5min              | X            | > 300 = 5min                        |




**OBSERVACIONES Y COMPARACIONES:**

Aquí disponemos de una tabla comparativa entre las soluciones de los algoritmos de búsqueda A* y B&B. En el eje de la Y observamos los diferentes tableros que hemos implementado. En el eje X observamos que las dos primeras columnas corresponden a la solución del algoritmo A* y a su derecha una tabla que expresa el tiempo que se tarda en ejecutar dicha sección de código. Lo mismo pasa con el algoritmo de B&B, tiene su columna de soluciones y tiempos de ejecución.

Reflejamos el tiempo en segundos (s) y la solución se expresa mediante el máximo nº de caballos posibles sin que se amenacen uno a otro que se pueden introducir en dicho tablero. Esa misma solución se expresa de forma gráfica en la salida de las ejecuciones.

Las casillas marcadas con X reflejan una ejecuión de tiempo excesivo donde no se halla ningún tipo de resultado.


**1. COMPARACIONES**
- En primer lugar observamos que el algoritmo A* realiza una búsqueda más extensiva que el algoritmo B&B. Llega hasta el tablero 5x5 de manera rápida y además hallando una solición. Cosa que B&B no llega a realizar ya que explorar sistemáticamente más posibilidades, lo que se traduce en tiempos de ejecución más largos y dificultad para resolver tableros grandes.

- Los tiempos de ejecución de A* son notablemente menores que los de B&B. Cogen una diferencia mayor a partir del tablero 3x5. A ese estado de ejecución A* es 54,7 veces más rápido que B&B.

- Ambos alogitmos estan destinados a la búsqueda del camino óptimo. Se diferencian en que B&B pertenece al grupo de algoritmos de búsqueda no informada (no usa heurística) y A* al grupo de búsiqueda informada (sí usa heurísitica).

**2. CONCLUSIONES:**

En todas las ejecuciones el algoritmo A* de búsqueda informada mostró más eficiencia ya que al emplear la heurística reduce considerablemente el número de nodos explorados y alcanza la solución en menos pasos y en un menor tiempo de ejecución. De modo que A* sería el algoritmo elegido en este tipo de problema con estas caraterísiticas particualres.

En relacion a la escalabilidad de los tableros, A* se comporta de manera más razonable y eficaz que B&B ya que este se ve obligado a explorar  exhaustivamente opciones en ausencia de una guía heurística.

En cuanto al entrono de la práctica queremos hacer hincapié en observaciones específicas de su funcionamiento:
- A medida que hemos realizado nuemerosas ejecuciones nos hemos dado cuenta de que con cada una de ellas bajaba el tiempo de ejecución de los algoitmos.